## `GPUDrive` simulator concepts

In this notebook, we demonstrate how to work with the `GPUDrive` simulator and access its basic attributes in Python. The simulator, written in C++, is built on top of the [Madrona Engine](https://madrona-engine.github.io/).

In [1]:
import os
import torch
from pathlib import Path
import gpudrive

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)


from pygpudrive.env.config import SceneConfig
from pygpudrive.env.scene_selector import select_scenes
scene_config = SceneConfig(path="data/processed/examples", num_scenes=1)

### Summary

- `GPUDrive` simulations are discretized traffic scenarios. A scenario is a constructed snapshot of traffic situation at a particular timepoint.
- The state of the vehicle of focus is referred to as the **ego state**. Each vehicle has their own partial view of the traffic scene; and a visible state is constructed by parameterizing the view distance of the driver. The **action** for each vehicle is a (1, 3) tuple with the acceleration, steering and head angle of the vehicle.
- The `step()` method advances the simulation with a desired step size. By default, the dynamics of vehicles are driven by a kinematic bicycle model. If a vehicle is not controlled (that is, we do not give it actions), its position, heading, and speed will be updated according to a the human expert demonstrations.


### Instantiating a sim object with default parameters

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
sim = gpudrive.SimManager(
    exec_mode=gpudrive.madrona.ExecMode.CUDA
    if device == "cuda"
    else gpudrive.madrona.ExecMode.CPU,
    gpu_id=0,
    scenes=select_scenes(scene_config),
    params=gpudrive.Parameters(),  # Environment parameters
)


--- Ratio unique scenes / number of worls = 1 / 1 ---



The simulator provides the following functions:
- `reset(world_idx)` resets a specific world or environment at the given index.

In [4]:
sim.reset([0])

- `step()` advances the dynamics of all worlds.

In [5]:
sim.step()

### Exporting tensors

To retrieve a tensor from the simulator, call the specific `tensor()` method, followed by either `to_torch()` or `to_jax()`.

For example, here is how to access the ego state, or self-observation tensor:

In [6]:
observation_tensor = sim.self_observation_tensor().to_torch()

observation_tensor.shape, observation_tensor.device

(torch.Size([1, 128, 7]), device(type='cpu'))

Or alternatively:

In [7]:
observation_tensor_jax = sim.self_observation_tensor().to_jax()

observation_tensor_jax.shape, observation_tensor_jax.devices()

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


((1, 128, 7), {CpuDevice(id=0)})

Here are all available tensor exports and methods on the sim object:

In [8]:
for attr in dir(sim):
    if not attr.startswith("_"):
        print(attr)

absolute_self_observation_tensor
action_tensor
agent_roadmap_tensor
controlled_state_tensor
depth_tensor
done_tensor
expert_trajectory_tensor
info_tensor
lidar_tensor
map_observation_tensor
partner_observations_tensor
reset
response_type_tensor
reward_tensor
rgb_tensor
self_observation_tensor
shape_tensor
step
steps_remaining_tensor
valid_state_tensor


### Inspect valid and controlled agents

To check the number of agents and road points in each world, you can use the `shape_tensor`.

The shape tensor is a 2D tensor where the first dimension represents the number of worlds, and the second dimension represents the shape of each world.

In [9]:
shape_tensor = sim.shape_tensor().to_jax()
print(f"Shape tensor has a shape of (Num Worlds, 2): {shape_tensor.shape}")

for world_idx in range(shape_tensor.shape[0]):
    print(
        f"World {world_idx} has {shape_tensor[world_idx][0]} VALID agents and {shape_tensor[world_idx][1]} VALID road objects"
    )

Shape tensor has a shape of (Num Worlds, 2): (1, 2)
World 0 has 10 VALID agents and 3195 VALID road objects


The number of **valid** agents indicates the number of controllable agents (vehicles). Some vehicles or bicycles may be initialized in incorrect positions or remain static; these are marked as **invalid** and cannot be controlled.

The sim comes with a mask that indicates which agents can be controlled. Entries are `1` for agents that can be controlled, and `0` otherwise.

In [10]:
controlled_state_tensor = sim.controlled_state_tensor().to_torch()
print(
    "Controlled state tensor has a shape of (num_worlds, max_num_agents_in_scene, 1): ",
    controlled_state_tensor.shape,
)

Controlled state tensor has a shape of (num_worlds, max_num_agents_in_scene, 1):  torch.Size([1, 128, 1])


In [11]:
# We can control 3 agents in this world
controlled_state_tensor.squeeze()

tensor([1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)

In [12]:
controlled_state_tensor.sum().item()

6

### Actions

The action space consists of three types:

- **Acceleration**: Continuous float values representing the acceleration applied to the agents. This affects how quickly an agent speeds up or slows down.
- **Steering Angle**: Continuous float values representing the steering angle, following the bicycle kinematic model. This determines how sharply an agent turns.
- **Heading Angle** (currently unused): Continuous float values for the heading angle, which control the direction an agent is facing.


The action tensor stores the current actions for all agents across all worlds:

In [13]:
action_tensor = sim.action_tensor().to_torch()
print(
    f"Action tensor has a shape of (num_worlds, max_num_agents_in_scene, 3): {action_tensor.shape}"
)

Action tensor has a shape of (num_worlds, max_num_agents_in_scene, 3): torch.Size([1, 128, 10])


To set the actions for all controlled agents, we use the `copy_()` method:

In [14]:
actions_tensor = sim.action_tensor().to_torch()

actions = torch.full(actions_tensor.shape, 1.0)
actions_tensor.copy_(actions)

print(f"Actions tensor after setting all actions to 1: {actions_tensor[0][0]}")

# Call step() to apply the actions
sim.step()

Actions tensor after setting all actions to 1: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


### Inspecting the simulator settings

In [15]:
params = gpudrive.Parameters()

print("Parameters:")
for attr in dir(params):
    if not attr.startswith("__"):
        value = getattr(params, attr)
        print(f"{attr:20}: {value}")
        if attr == "rewardParams":
            print("Reward parameters:")
            reward_params = getattr(params, attr)
            for attr2 in dir(reward_params):
                if not attr2.startswith("__"):
                    value2 = getattr(reward_params, attr2)
                    print(f"    {attr2:18}: {value2}")

Parameters:
IgnoreNonVehicles   : False
collisionBehaviour  : gpudrive.CollisionBehaviour.AgentStop
disableClassicalObs : False
dynamicsModel       : gpudrive.DynamicsModel.Classic
enableLidar         : False
initOnlyValidAgentsAtFirstStep: True
isStaticAgentControlled: False
maxNumControlledAgents: 10000
observationRadius   : 0.0
polylineReductionThreshold: 0.0
rewardParams        : <gpudrive.RewardParams object at 0x7f3a8c4a8670>
Reward parameters:
    distanceToExpertThreshold: 0.0
    distanceToGoalThreshold: 0.0
    rewardType        : gpudrive.RewardType.DistanceBased
roadObservationAlgorithm: gpudrive.FindRoadObservationsWith.KNearestEntitiesWithRadiusFiltering


### Configuring the simulator 

To set the parameters of the simulator, fill in the values for each attribute of the parameter object as below. This allows you to customize the simulation settings.

The params object can be passed to the sim constructor like this:

```Python
sim = gpudrive.SimManager(
    ...
    params=params 
)
```

See our [README](https://github.com/Emerge-Lab/gpudrive/tree/main?tab=readme-ov-file#configuring-the-sim) for the full documentation.

In [16]:
reward_params = gpudrive.RewardParams()
reward_params.rewardType = gpudrive.RewardType.DistanceBased
reward_params.distanceToGoalThreshold = 1.0
reward_params.distanceToExpertThreshold = 1.0

# Initialize Parameters
params = gpudrive.Parameters()
params.polylineReductionThreshold = 1.0
params.observationRadius = 100.0
params.collisionBehaviour = gpudrive.CollisionBehaviour.Ignore
params.maxNumControlledAgents = 10
params.rewardParams = reward_params

### Running an episode of the sim

Putting everything together, the full interaction loop looks like this:

In [18]:
sim.reset([0])

actions_shape = sim.action_tensor().to_torch().shape
dones = sim.done_tensor().to_torch()

while not torch.all(sim.done_tensor().to_torch()):
    obs, rews, dones = (
        sim.self_observation_tensor().to_torch(),
        sim.reward_tensor().to_torch(),
        sim.done_tensor().to_torch(),
    )
    actions = torch.rand(actions_shape)
    sim.action_tensor().to_torch().copy_(actions)
    sim.step()

### [optional] Detailed documentation for simulator configurations 📜

This section provides detailed information about the observation tensors, rewards, road reduction algorithm, collision behavior, and various other parameters used in the simulator.

#### Observation Space

**SelfObservation**

The `SelfObservation` tensor of shape `(5,)` for each agent provides information about the agent's own state. The respective values are:

- `SelfObservation[0]`: Represents the current *speed* of the agent.
- `SelfObservation[1:3]`: *Length* and *width* of the agent.
- `SelfObservation[3:5]`: *Coordinates (x,y)* of the goal relative to the agent.
- `SelfObservation[5]`: Represents if the agent has collided. Values in `{0,1}`.

**MapObservation**

The `MapObservation` tensor of shape `(4,)` for each agent provides the *absolute* position of map objects. The values are:

- `MapObservation[0:2]`: Represents the position of the `MapObject`.
- `MapObservation[2:5]`: Represents the scale of the `MapObject` in terms of length, width, and height.
- `MapObservation[5]`: Represents the heading angle of the `MapObject`.
- `MapObservation[6]`: Represents the type of the `MapObject`.

**PartnerObservation**

The `PartnerObservation` tensor of shape `(num_agents-1, 7)` for each agent provides information about other agents within the `params.observationRadius`. All the values in this tensor are *relative to the ego agent*. The respective values for each `PartnerObservation` are:

- `PartnerObservation[0]`: The *speed* of the observed neighboring agent.
- `PartnerObservation[1:3]`: The *position (x,y)* of the observed neighboring agent.
- `PartnerObservation[3]`: The *orientation* of the neighboring agent.
- `PartnerObservation[4:6]`: The *length* and *width* of the neighboring agent.
- `PartnerObservation[6]`: The type of agent.

**AgentMapObservations**

The `AgentMapObservations` tensor of shape `(num_road_objs, 7)` for each agent provides information about the road objects within the `params.observationRadius`. All the values in this tensor are *relative to the ego agent*. The respective values for each `AgentMapObservations` are the same as `MapObservations`.

#### Rewards

* `RewardType`: There are three types of rewards that can be exported from the simulator:
  - `DistanceBased`: Exports the distance of the agent to the goal.
  - `OnGoalAchieved`: Exports 1 if the agent has reached the goal, else 0.
  - `Dense` (Not Implemented): Exports the distance of the agent from its expert trajectory specified in the dataset.
* `distanceToGoalThreshold`: This threshold is used to determine if the agent has reached the goal or not. `Default: 0.0`.
* `distanceToExpertThreshold`: This threshold is used to determine if the agent is following the expert trajectory or not. `Default: 0.0`.

#### Road Reduction Algorithm

To manage performance and simplify the observation space of the simulator, we apply a polyline reduction algorithm on the road edges, lines, and lanes. We use the ['Visvalingam-Whyatt Algorithm'](https://en.wikipedia.org/wiki/Visvalingam%E2%80%93Whyatt_algorithm).

* `polylineReductionThreshold`: This threshold determines how much reduction is to be applied to the road lines. Ranges from `0` to `+ve inf`. If set to `0`, no reduction will be applied. `Default: 0.5`.

#### Collision Behaviour

For easy troubleshooting and learning various policies, the behavior of the agents on collisions can be configured.

* `AgentStop`: The agents in collision would simply stop at the point of collision. No further actions will be applied to these agents.
* `AgentRemoved`: The agents in collision would simply be removed from the scene.
* `Ignore`: The agents in collision still output that they collided, but they will continue to move around as if they did not collide.

#### Misc Params

* `ObservationRadius`: Defines the radius within which an agent can observe its surroundings. Objects outside the `ObservationRadius` are zeroed out in the observations.
* `MaxNumControlledVehicles`: Controls the maximum number of agents that can be controlled in the simulator. If a particular file has fewer valid agents, some worlds may have fewer controlled agents. We pick the first `MaxNumControlledVehicles` **valid** agents to control, and the rest are controlled via their expert trajectories.
* `IgnoreNonVehicles`: Defines the policy of not initializing pedestrians/cyclists. Default: `false`.
* `roadObservationAlgorithm`: Choose between `KNearestEntitiesWithRadiusFiltering` and `AllEntitiesWithRadiusFiltering`. The `KNearestEntitiesWithRadiusFiltering` filters out `kMaxAgentMapObservationsCount` nearest points within the `observationRadius` of the agents, while `AllEntitiesWithRadiusFiltering` runs a linear search in the same radius. Default: `KNearestEntitiesWithRadiusFiltering`.
* `initOnlyValidAgentsAtFirstStep`: Controls if only agents valid at the first step are initialized into the simulator. Default: `true`.
* `isStaticAgentControlled`: Controls if agents like parked vehicles that are already at their goals should be allowed to be controlled or set as static. Default: `false`.
* `enableLidar`: Enables lidar observations.
* `disableClassicalObs`: Disables setting `PartnerObservations` and `AgentMapObservations`. Generally used to speed up the simulator if lidar observations are enabled and the above observations are not used. Default: `false`.
* `useWayMaxModel`: Sets if the WayMax dynamics model should be used. Default: `false`.

#### Types of Objects

* Road types:
  - `RoadEdge`
  - `RoadLine`
  - `RoadLane`
  - `CrossWalk`
  - `SpeedBump`
  - `StopSign`
* Agent types:
  - `Vehicle`
  - `Pedestrian`
  - `Cyclist`
* `Padding` type: A special type used to pad entities in different worlds to ensure consistent output shapes.
* `None` type: A special type marking entities as invalid and should not be considered for learning. This can arise if an entity is outside the `ObservationRadius` or if the entity collided and the collision behavior is set to `AgentRemoved`.
